<a href="https://colab.research.google.com/github/BigTMiami/AdaptOrDie/blob/main/Amazon_Helpfulness_Classification_on_5M_pretrained_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Questions

* Does evaluation use the Test or Dev set?
*   Settings for the text classifier vs the domain training are not 100% clear
* Are we going to use seeds to average 5 runs?



# Changes from previous run
* Added google drive for checkpoint saves
* Did not install huggingface_hub - not needed
* Added print_gpu_utilization for checking memory after each major step
* Added overall time for script to run
* Moved imports to code block where first used
* Removed numpy import - isn't used anymore
* Load config from BigTMiami/amazon_pretraining_5M_model
* Training Arguements
  * Output dir = amazon_helpfulness_clas_on_amazon_5M_model
  * torch_compile=True,  # Much Faster
  * hub_strategy="checkpoint", # Only pushes at end with save_model()
  * load_best_model_at_end=False, # Set to false - we want the last trained model like the paper
  * torch_compile=True,  # Much Faster
  * logging_steps=500, # Logs training progress
  * save_total_limit=2, # Saves latest 2 checkpoints

In [1]:
from time import time
start_time = time()

1712672867.741167


In [2]:
from google.colab import drive
drive.mount("/content/drive")
%cd '/content/drive/MyDrive/AdaptOrDie'

Mounted at /content/drive
/content/drive/MyDrive/AdaptOrDie


In [3]:
!pip install transformers[torch]
!pip install datasets
!pip install scikit-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.4/297.4 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 70.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 58.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 93.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 30.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━

In [ ]:
!pip install pynvml

In [6]:
from pynvml import *

def print_gpu_utilization():
    nvmlInit()
    handle = nvmlDeviceGetHandleByIndex(0)
    info = nvmlDeviceGetMemoryInfo(handle)
    gpu_used = info.used//1024**2
    print(f"GPU {gpu_used} MB")

print_gpu_utilization()

GPU 448 MB


In [7]:
from sklearn.metrics import accuracy_score,  f1_score

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)

    # Calculate accuracy
    accuracy = accuracy_score(labels, preds)

   # Calculate precision, recall, and F1-score
    f1 = f1_score(labels, preds, average='macro')

    return {
        'accuracy': accuracy,
        'f1_macro': f1
    }

In [8]:
from datasets import load_dataset

dataset = load_dataset("BigTMiami/amazon_helpfulness")
dataset
print_gpu_utilization()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/repocard.py:105: UserWarning: Repo card metadata block was not found. Setting CardData to empty.
  warnings.warn("Repo card metadata block was not found. Setting CardData to empty.")


Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Generating dev split: 0 examples [00:00, ? examples/s]

GPU 448 MB


In [9]:
from transformers import AutoTokenizer, DataCollatorWithPadding

tokenizer = AutoTokenizer.from_pretrained("roberta-base")

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

print_gpu_utilization()

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

GPU 448 MB


In [10]:
#Label Settings
id2label = {0: "unhelpful", 1: "helpful"}
label2id = {"unhelpful": 0, "helpful": 1}

In [11]:
from transformers import AutoConfig

# Set Classifier Settings
# USING BigTMiami/amazon_pretraining_5M_model instead of roberta-base
classification_config = AutoConfig.from_pretrained('BigTMiami/amazon_pretraining_5M_model')
classification_config.classifier_dropout = 0.1 # From Paper
classification_config.num_of_labels = 2
classification_config.id2label=id2label
classification_config.label2id=label2id,
classification_config

config.json:   0%|          | 0.00/672 [00:00<?, ?B/s]

RobertaConfig {
  "_name_or_path": "BigTMiami/amazon_pretraining_5M_model",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": 0.1,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "unhelpful",
    "1": "helpful"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": [
    {
      "helpful": 1,
      "unhelpful": 0
    }
  ],
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "num_of_labels": 2,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "torch_dtype": "float32",
  "transformers_version": "4.38.2",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 50265
}

In [12]:
from transformers import AutoModelForSequenceClassification

# Changed to BigTMiami/amazon_pretraining_5M_model from roberta-base
model = AutoModelForSequenceClassification.from_pretrained(
    "BigTMiami/amazon_pretraining_5M_model",  config=classification_config
)

print_gpu_utilization()

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at BigTMiami/amazon_pretraining_5M_model and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


GPU 448 MB


In [17]:
from transformers import TrainingArguments

# Changed output dir
# Added torch compile

training_args = TrainingArguments(
    output_dir="amazon_helpfulness_clas_on_amazon_5M_model",
    overwrite_output_dir=True,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5, # Paper: this is for Classification, not domain training
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3, # NEED TO TRAIN FOR REAL TEST
    weight_decay=0.01,
    warmup_ratio=0.06, # Paper: warmup proportion of 0.06
    adam_epsilon=1e-6, # Paper 1e-6 (huggingface default 1e-08)
    adam_beta1=0.9, # Paper: Adam weights 0.9
    adam_beta2=0.98, # Paper: Adam weights 0.98 (huggingface default  0.999)
    lr_scheduler_type="linear",
    save_total_limit=2, # Saves latest 2 checkpoints
    push_to_hub=True,
    hub_strategy="checkpoint", # Only pushes at end with save_model()
    load_best_model_at_end=False, # Set to false - we want the last trained model like the paper
    torch_compile=True,  # Much Faster
    logging_steps=500, # Logs training progress
)

In [18]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)
trainer.args

print_gpu_utilization()

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


GPU 1399 MB


In [19]:
results = trainer.train()
results
print_gpu_utilization()

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Epoch,Training Loss,Validation Loss,Accuracy,F1 Macro
1,0.414100,0.419914,0.853400,0.460451
2,0.417500,0.424494,0.853400,0.460451
3,0.422200,0.417842,0.853400,0.460451


GPU 19619 MB


In [20]:
eval_results = trainer.evaluate()
eval_results
print_gpu_utilization()

GPU 19619 MB


In [21]:
trainer.push_to_hub()

events.out.tfevents.1712678289.d6a593d959fc.1100.1:   0%|          | 0.00/470 [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/BigTMiami/amazon_helpfulness_clas_on_amazon_5M_model/commit/c7643e8a6885a9983b045575ed8a68e8c90a4492', commit_message='End of training', commit_description='', oid='c7643e8a6885a9983b045575ed8a68e8c90a4492', pr_url=None, pr_revision=None, pr_num=None)

In [23]:
end_time = time()
total_time = end_time - start_time
print(f"Total Script Runtime:{total_time:.0f} in seconds")
print(f"Total Script Runtime:{total_time / 60:.0f} in minutes")
print(f"Total Script Runtime:{total_time / 3600:.1f} in hours")




Total Script Runtime:7846 in seconds
Total Script Runtime:131 in minutes
Total Script Runtime:2.2 in hours


In [25]:
print(results)

TrainOutput(global_step=21612, training_loss=0.42137658664435784, metrics={'train_runtime': 2978.5315, 'train_samples_per_second': 116.082, 'train_steps_per_second': 7.256, 'total_flos': 8.478116382722165e+16, 'train_loss': 0.42137658664435784, 'epoch': 3.0})


In [24]:
print(eval_results)

{'eval_loss': 0.417842298746109, 'eval_accuracy': 0.8534, 'eval_f1_macro': 0.46045106291140603, 'eval_runtime': 63.2068, 'eval_samples_per_second': 395.527, 'eval_steps_per_second': 24.728, 'epoch': 3.0}


In [ ]:
print("Disconnecting Session")
from google.colab import runtime
runtime.unassign()
